## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [11]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense, Activation
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


In [21]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [23]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

In [ ]:

classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32,(3,3),padding='same',input_shape=x_train.shape[1:]))
classifier.add(BatchNormalization(momentum=0.99, epsilon=0.001))
classifier.add(Activation('relu'))


'''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(64,(3,3),padding='same'))
classifier.add(BatchNormalization(momentum=0.99, epsilon=0.001))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(128)) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(10,activation='relu'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)

Epoch 1/100
50000/50000 [==============================] - 116s 2ms/step - loss: 2.2139 - accuracy: 0.2252
Epoch 2/100
50000/50000 [==============================] - 115s 2ms/step - loss: 1.8299 - accuracy: 0.4092
Epoch 3/100
50000/50000 [==============================] - 117s 2ms/step - loss: 2.1229 - accuracy: 0.3404
Epoch 4/100
50000/50000 [==============================] - 119s 2ms/step - loss: 2.2078 - accuracy: 0.1798
Epoch 5/100
50000/50000 [==============================] - 114s 2ms/step - loss: 2.0288 - accuracy: 0.2320
Epoch 6/100
28300/50000 [===============>..............] - ETA: 50s - loss: 1.9676 - accuracy: 0.2670

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [18]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[0.80643475]], dtype=float32)